In [ ]:
from nBestAligner.nBestAlign import align
import torch

In [ ]:
nbest = [
    [1,2,3,4,6],
    [0, 1, 2, 10],
    [2,3,5,7],
    [5,10, 4, 8],
]

In [ ]:
device = 'cuda' if  torch.cuda.is_available() else 'cpu'

In [ ]:
device = torch.device(device)

In [ ]:
align(nbest, 4)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer(
                "cyclone/simcse-chinese-roberta-wwm-ext",
                device = device
            )


In [ ]:
tokens = ['广州市房地产中介协会分析'].to(device)

In [ ]:
embedding = model.encode(tokens)

In [ ]:
embedding.shape

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
token = tokenizer.convert_tokens_to_ids(list('广州市房地产中介协会分析'))

In [ ]:
model.encode(token)

In [1]:
import torch
import logging
from torch.nn.functional import log_softmax
from transformers import (
    BertForMaskedLM,
    AutoModelForCausalLM,
    BertTokenizerFast,
)
from torch.optim import AdamW

In [2]:
model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese')

In [3]:
seq = torch.tensor([
    [
                101,
                2408,
                2336,
                2356,
                2791,
                1765,
                772,
                704,
                792,
                1291,
                833,
                1146,
                3358,
                102
            ],
            [
                101,
                2408,
                2336,
                2356,
                2791,
                1765,
                772,
                704,
                6381,
                1291,
                833,
                102,
                0,
                0
            ]
        ])

In [4]:
output = model(seq)

In [5]:
output.logits.shape

torch.Size([2, 14, 21128])

In [6]:
non_cls_index = (seq != 101)
non_sep_index = (seq != 102)
non_pad_index = (seq != 0)

In [8]:
temp_logit = torch.logical_and(non_cls_index, non_sep_index)
token_index = torch.logical_and(temp_logit, non_pad_index)
token_index.shape

torch.Size([2, 14])

In [9]:
real_token = seq[token_index]

In [21]:
seq[token_index]

tensor([2408, 2336, 2356, 2791, 1765,  772,  704,  792, 1291,  833, 1146, 3358,
        2408, 2336, 2356, 2791, 1765,  772,  704, 6381, 1291,  833])

In [10]:
logit = output.logits

In [11]:
logit.shape

torch.Size([2, 14, 21128])

In [12]:
seq.shape

torch.Size([2, 14])

In [14]:
score = logit.gather(2, seq.unsqueeze(2)).squeeze(-1)
score.shape

torch.Size([2, 14])

In [26]:
true_index = torch.nonzero(token_index)

In [27]:
sum_score = 0
last_i = -1
result = []
for i, j in true_index:
    if (last_i >= 0 and last_i != i):
        print('flush')
        result.append(sum_score)
        sum_score = 0
    print(i)
    sum_score += score[i][j].item()
    last_i = i.item()
result.append(sum_score)

tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
flush
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)
tensor(1)


In [139]:
result

[2.821585178375244, -20.840754985809326]

In [36]:
exp_logit = torch.exp(logit)

In [38]:
torch.sum(logit, -1)

tensor([-127098.0000, -141032.8750, -140960.9844, -145216.1875, -131136.9375,
        -124833.8438, -140478.4844, -138558.0000, -148117.0469, -142772.9219,
        -141909.0938, -144588.3281, -147351.7812, -148185.6562],
       grad_fn=<SumBackward1>)

In [ ]:
string = "你好啊"
list(string)

In [1]:
from transformers import BertTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained('fnlp/bart-base-chinese')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [3]:
string1 = '我要吃飯要吃飯吃飯飯'
string2 = '我要吃飯#要吃飯#吃飯#飯'

In [11]:
str_list1 = [x for x in string1]
str_list2 = [x for x in string2]

In [9]:
tokenizer.convert_ids_to_tokens([0])

['[PAD]']

In [19]:
print(str_list1)
print(str_list2)

['我', '要', '吃', '飯', '要', '吃', '飯', '吃', '飯', '飯']
['我', '要', '吃', '飯', '#', '要', '吃', '飯', '#', '吃', '飯', '#', '飯']


In [18]:
print(tokenizer.tokenize(string1))
print(tokenizer.tokenize(string2))

['我', '要', '吃', '飯', '要', '吃', '飯', '吃', '飯', '飯']
['我', '要', '吃', '飯', '#', '要', '吃', '飯', '#', '吃', '飯', '#', '飯']


In [21]:
str_list2 == tokenizer.tokenize(string2)

True

In [14]:
tokenizer.convert_tokens_to_ids(str_list2)

[2769, 6206, 1391, 7613, 108, 6206, 1391, 7613, 108, 1391, 7613, 108, 7613]

In [ ]:
string = "00010002"
string = string.lstrip('0')
print(string)
split_string = [string[max(i - 4, 0) : i] for i in range(len(string), 0 , -4)]
split_string[::-1]